In [16]:
#!pip install recordlinkage==0.14
import recordlinkage
import pandas as pd
import recordlinkage as rl 
from recordlinkage.index import Block
from recordlinkage.algorithms.distance import _haversine_distance
from recordlinkage.algorithms.numeric import _linear_sim
import random
import pickle 
import numpy as np

In [2]:
#Reading Data
df_clean = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_red.csv')
#There was a duplicated entry in ListingId so delete
df_clean[df_clean.duplicated(subset=['ListingId']) == True]
df_clean.drop(132,axis=0,inplace=True)
df_clean[df_clean['ListingId']==10217568]


#To have tuples with LitingsId as index ,useful for later processing
df_tuple = df_clean.set_index('ListingId')
# To compare Harversine we need lat and ln in a tuple
df_clean['Lat_Lng'] =list(zip(df_clean.Lat, df_clean.Lng))

In [76]:
#Read the feature embeddings df
df_uns = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_uns_label_img_red_hot.csv')
#Stratified random sampling needs binning the agg score of both classes
df_uns_high = df_uns[df_uns['agg_score'] > 0.50]
df_uns_low = df_uns[df_uns['agg_score'] < 0.50]

In [77]:
# Indexes which may be selected
top_pairs = list(zip(df_uns_high['ListingId_1'],df_uns_high['ListingId_2']))
bottom_pairs = list(zip(df_uns_low['ListingId_1'],df_uns_low['ListingId_2']))

In [85]:
#Sampling
rng =  np.random.default_rng(seed=2023)
top_idx =  rng.integers(low=0, high=len(top_pairs), size=100)
bottom_idx = rng.integers(low=0, high=len(bottom_pairs), size=100)
#convert to arrays to select with array index
top_pairs = np.array(top_pairs)
bottom_pairs = np.array(bottom_pairs)
#Slect random chosen index
sample_top = top_pairs[top_idx]
sample_bottom = bottom_pairs[bottom_idx]

In [89]:
df_clean_1 = df_clean.drop(['Host_picture_url','LaBSE_emb', 'Lat_Lng','Host_thumbnail_url'],axis=1)
labels = []

In [90]:
#Labeling loop
idx = 0
for id_1,id_2 in sample_top:
    print('Label:',idx)
    url_1 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_1]]
    url_2 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_2]]
    print("Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping")
    print(url_1[0])
    print(url_2[0])
    label = int(input())
    if label == 1 or label==0:
        labels.append(((id_1,id_2),label))
        print('Saved!')
    if label == 9:
        break
    if label == 3:
        lista = [print(x,':',y,'==',z) for x,y,z in zip(df_clean_1.columns.tolist(),
                                                        df_clean_1[df_clean_1['ListingId'] == id_1].values.tolist()[0],
                                                        df_clean_1[df_clean_1['ListingId'] == id_2].values.tolist()[0])]
        label = int(input())
        labels.append(((id_1,id_2),label))
        print('Saved!')
    idx += 1

#Labeling loop
idx =0
for id_1,id_2 in sample_bottom:
    print('Label:',idx)
    url_1 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_1]]
    url_2 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_2]]
    print("Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping")
    print(url_1[0])
    print(url_2[0])
    label = int(input())
    if label == 1 or label==0:
        labels.append(((id_1,id_2),label))
        print('Saved!')
    if label == 9:
        break
    if label == 3:
        lista = [print(x,':',y,'==',z) for x,y,z in zip(df_clean_1.columns.tolist(),
                                                        df_clean_1[df_clean_1['ListingId'] == id_1].values.tolist()[0],
                                                        df_clean_1[df_clean_1['ListingId'] == id_2].values.tolist()[0])]
        label = int(input())
        labels.append(((id_1,id_2),label))
        print('Saved!')
    idx +=1


Label: 0
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/41710302
https://www.airbnb.nl/rooms/31945182
3
site : AIRBNB == AIRBNB
ListingId : 41710302 == 31945182
Name : private single ensuite == small twin ensuite
postalcode : 1012 JA == 1012 JA
Bedrooms : 1.0 == 1.0
Bathrooms : 1.0 == 1.0
Beds : 1.0 == 1.0
Person_capacity : 1 == 2
Host_first_name : st christophers at the winston == st christophers at the winston
Instant_bookable : 1 == 1
Is_new_listing : 0 == 0
Neighborhood : amsterdam centrum == amsterdam centrum
Lat : 52.37392 == 52.37392
Lng : 4.89635 == 4.89635
Picture_url : https://a0.muscache.com/im/pictures/31b67226-c1e7-4b6f-b9a7-a4ac1eaad1da.jpg?aki_policy=large == https://a0.muscache.com/im/pictures/a6d9b2aa-0a0d-462b-a00a-5bd382514721.jpg?aki_policy=large
reviews_count : 0.0 == 91.0
Room_type_category : private_room == private_room
Room_type : privékamer == privékamer
Space_type : pr

1
Saved!
Label: 1
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/44423961
https://www.airbnb.nl/rooms/44310953
1
Saved!
Label: 2
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/16203884
https://www.airbnb.nl/rooms/16203297
1
Saved!
Label: 3
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/42008190
https://www.airbnb.nl/rooms/14572858
3
site : AIRBNB == AIRBNB
ListingId : 42008190 == 14572858
Name : comfy room for female travellers == 2bdrm  central  spacious 100m2
postalcode : 1094 AB == 1094 CB
Bedrooms : 1.0 == 2.0
Bathrooms : 15.0 == 15.0
Beds : 1.0 == 2.0
Person_capacity : 2 == 4
Host_first_name : nicolette == nicolette
Instant_bookable : 0 == 0
Is_new_listing : 0 == 0
Neighborhood : amsterdam oost == amsterdam

0
Saved!
Label: 4
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/20791949
https://www.airbnb.nl/rooms/20433472
1
Saved!
Label: 5
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/52874661
https://www.airbnb.nl/rooms/52873591
1
Saved!
Label: 6
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/14159545
https://www.airbnb.nl/rooms/12213244
0
Saved!
Label: 7
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/38452521
https://www.airbnb.nl/rooms/37732527
1
Saved!
Label: 8
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/5722615
https://www.airbnb

1
Saved!
Label: 19
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/15038670
https://www.airbnb.nl/rooms/12940835
0
Saved!
Label: 20
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/34242751
https://www.airbnb.nl/rooms/25021801
0
Saved!
Label: 21
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/44534876
https://www.airbnb.nl/rooms/21687625
3
site : AIRBNB == AIRBNB
ListingId : 44534876 == 21687625
Name : cosy houseboat in the city center for couples == canal house next to amsterdam cs  perfect for fam 
postalcode : 1011 CN == 1011 BG
Bedrooms : 1.0 == 3.0
Bathrooms : 1.5 == 1.0
Beds : 1.0 == 6.0
Person_capacity : 2 == 4
Host_first_name : nina == friederike
Instant_bookable : 0 == 0
Is_new_listing : 0 == 0
Neighborhood

0
Saved!
Label: 22
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/25339698
https://www.airbnb.nl/rooms/25336232
3
site : AIRBNB == AIRBNB
ListingId : 25339698 == 25336232
Name : vincent van gogh apartment garden view   access == frans hals apartment garden view
postalcode : 1017 EN == 1017 EN
Bedrooms : 1.0 == 1.0
Bathrooms : 1.0 == 1.0
Beds : 1.0 == 1.0
Person_capacity : 2 == 2
Host_first_name : anja == anja
Instant_bookable : 0 == 0
Is_new_listing : 0 == 0
Neighborhood : amsterdam centrum == amsterdam centrum
Lat : 52.3643 == 52.3643
Lng : 4.8886699999999985 == 4.8886699999999985
Picture_url : https://a0.muscache.com/im/pictures/fc358985-df39-4dad-9b4a-8fe41351e1fa.jpg?aki_policy=large == https://a0.muscache.com/im/pictures/641eb7a4-0d6d-45fb-9219-b02f1fb739ca.jpg?aki_policy=large
reviews_count : 1.0 == 2.0
Room_type_category : entire_home == entire_home
Room_type : gehele woning appartement 

...

0
Saved!
Label: 96
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/1156220
https://www.airbnb.nl/rooms/10136640
3
site : AIRBNB == AIRBNB
ListingId : 1156220 == 10136640
Name : beautiful designed apartment in old school   west == appartement with view  oud west  old west 
postalcode : 1053 AR == 1053 VN
Bedrooms : 4.0 == 1.0
Bathrooms : 25.0 == 1.5
Beds : 4.0 == 2.0
Person_capacity : 4 == 2
Host_first_name : cristina == rogier
Instant_bookable : 0 == 0
Is_new_listing : 0 == 0
Neighborhood : amsterdam west == amsterdam west
Lat : 52.36874 == 52.36964
Lng : 4.86693 == 4.86856
Picture_url : https://a0.muscache.com/im/pictures/e379fb0a-9171-4a0a-b233-ef5eb5f4c02e.jpg?aki_policy=large == https://a0.muscache.com/im/pictures/e5b8a7a5-7401-4982-95ed-faad9cb00192.jpg?aki_policy=large
reviews_count : 4.0 == 11.0
Room_type_category : entire_home == entire_home
Room_type : gehele woning appartement == gehel

0
Saved!
Label: 97
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/24589263
https://www.airbnb.nl/rooms/10593371
3
site : AIRBNB == AIRBNB
ListingId : 24589263 == 10593371
Name : unique apartment  amazing canal view prinsengracht == woonboot in centrum amsterdam
postalcode : 1015 EB == 1015 RL
Bedrooms : 2.0 == 1.0
Bathrooms : 1.5 == 1.0
Beds : 2.0 == 1.0
Person_capacity : 3 == 2
Host_first_name : robert == janneke
Instant_bookable : 0 == 0
Is_new_listing : 0 == 0
Neighborhood : amsterdam centrum == amsterdam centrum
Lat : 52.37588 == 52.37675
Lng : 4.883769999999998 == 4.884530000000002
Picture_url : https://a0.muscache.com/im/pictures/2cf3b451-fc2a-408b-86a3-448c534224df.jpg?aki_policy=large == https://a0.muscache.com/im/pictures/354b13c2-3b5d-49c5-92e6-214e5a2f7512.jpg?aki_policy=large
reviews_count : 18.0 == 70.0
Room_type_category : entire_home == entire_home
Room_type : gehele woning appar

0
Saved!
Label: 98
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/29116941
https://www.airbnb.nl/rooms/21009564
3
site : AIRBNB == AIRBNB
ListingId : 29116941 == 21009564
Name : art nouveau houseboat overvieuwing amstel river == the little loft  private apartment in the center 
postalcode : 1011 JH == 1011 PN
Bedrooms : 2.0 == 1.0
Bathrooms : 1.0 == 1.0
Beds : 3.0 == 1.0
Person_capacity : 4 == 2
Host_first_name : rob == lenneke
Instant_bookable : 1 == 1
Is_new_listing : 0 == 0
Neighborhood : amsterdam centrum == amsterdam centrum
Lat : 52.36818 == 52.36778
Lng : 4.89915 == 4.89942
Picture_url : https://a0.muscache.com/im/pictures/640b5590-9d81-47f7-a740-2392e8541068.jpg?aki_policy=large == https://a0.muscache.com/im/pictures/228e0c9d-1e5e-44cf-9102-ab66750dbb03.jpg?aki_policy=large
reviews_count : 27.0 == 169.0
Room_type_category : entire_home == private_room
Room_type : gehele woning apparteme

0
Saved!
Label: 99
Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping
https://www.airbnb.nl/rooms/5494065
https://www.airbnb.nl/rooms/5427271
3
site : AIRBNB == AIRBNB
ListingId : 5494065 == 5427271
Name : centre apartment with private garden == newly renovated jordaan nine street
postalcode : 1016 TX == 1016 SH
Bedrooms : 1.0 == 2.0
Bathrooms : 1.0 == 1.0
Beds : 1.0 == 1.0
Person_capacity : 2 == 3
Host_first_name : alex == peter   david
Instant_bookable : 0 == 0
Is_new_listing : 0 == 0
Neighborhood : amsterdam centrum == amsterdam centrum
Lat : 52.36976 == 52.3703
Lng : 4.88065 == 4.8788300000000016
Picture_url : https://a0.muscache.com/im/pictures/87a32e9e-0a81-4006-aa59-5ca956d07919.jpg?aki_policy=large == https://a0.muscache.com/im/pictures/67741513/1c6ce542_original.jpg?aki_policy=large
reviews_count : 29.0 == 93.0
Room_type_category : entire_home == entire_home
Room_type : gehele woning appartement == gehele woning appa

0
Saved!


In [95]:
# Saving labels as pickle
file_name = "labels_no_hotels.pkl"
open_file = open(file_name, "wb")
pickle.dump(labels, open_file)
open_file.close()